In [1]:
import featuretools as ft
import pandas as pd
import numpy as np
from woodwork.logical_types import Categorical #import categorical type

import warnings
warnings.filterwarnings('ignore')


In [2]:
clients = pd.read_csv('../automated-feature-engineering/walk_through/data/clients.csv', parse_dates=['joined'])
loans = pd.read_csv('../automated-feature-engineering/walk_through/data/loans.csv', parse_dates=['loan_start', 'loan_end'])
payments = pd.read_csv('../automated-feature-engineering/walk_through/data/payments.csv', parse_dates=['payment_date'])

In [3]:
clients['join_month'] = clients['joined'].dt.month #parse date and derive feature by date
clients['log_income'] = np.log(clients['income']) #make income log scale
clients.head()


,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


In [4]:
stats = loans.groupby('client_id')['loan_amount'].agg(['sum']) #group by client_id and find total sum of loan
stats.columns = ['total_loan_amount']
stats.head()

,total_loan_amount
client_id,
25707,159279
26326,116321
26695,140845
26945,106889
29841,176634


In [5]:
clients = clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(10) #merge feature with clients

In [6]:
es = ft.EntitySet(id='client') #define entity set by featuretools


In [7]:
es = es.add_dataframe(dataframe_name='clients',
                      dataframe=clients,
                      index='client_id',
                      time_index='joined')

es = es.add_dataframe(dataframe_name='loans',
                      dataframe=loans,
                      index='loan_id',
                      time_index='loan_start',
                      logical_types={
                          'repaid': Categorical #specify repaid with categorical type
                      })

es = es.add_dataframe(dataframe_name='payments',
                      dataframe=payments,
                      make_index=True, #set index payment_id
                      index='payment_id',
                      time_index='payment_date',
                      logical_types= {
                          'missed': Categorical # specify missed with Categorical type
                      })

In [8]:
es['payments'].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
payment_id,Integer,['index']
loan_id,Integer,['numeric']
payment_amount,Integer,['numeric']
payment_date,Datetime,['time_index']
missed,Categorical,['category']


In [9]:
es['clients'].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
client_id,Integer,['index']
joined,Datetime,['time_index']
income,Integer,['numeric']
credit_score,Integer,['numeric']
join_month,Integer,['numeric']
log_income,Double,['numeric']
total_loan_amount,Integer,['numeric']


In [10]:
es = es.add_relationship('clients', 'client_id', 'loans', 'client_id') #add relationship
es = es.add_relationship('loans', 'loan_id', 'payments', 'loan_id')

In [11]:
es

Entityset: client
  DataFrames:
    clients [Rows: 10, Columns: 7]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

In [12]:
features, feature_names = ft.dfs(entityset=es,
                                 target_dataframe_name='clients',
                                 agg_primitives=['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives=['year', 'month', 'subtract_numeric_scalar','divide_numeric_scalar'])

In [13]:
pd.DataFrame(features['MONTH(joined)'])

,MONTH(joined)
client_id,
46180,11
46109,4
35214,8
32726,5
25707,10
49545,11
48177,6
35089,3
39505,10


In [14]:
pd.DataFrame(features['MEAN(payments.payment_amount)'])

,MEAN(payments.payment_amount)
client_id,
46180,1186.550336
46109,1375.560284
35214,1076.987952
32726,944.298013
25707,1178.552795
49545,1481.188482
48177,1199.715190
35089,1094.748428
39505,1162.026490


In [15]:
pd.DataFrame(features) #check the features and value

,income,credit_score,join_month,log_income,total_loan_amount,LAST(loans.loan_amount),LAST(loans.loan_id),LAST(loans.loan_type),LAST(loans.rate),LAST(loans.repaid),...,LAST(payments.payment_id) - 0,MAX(loans.loan_amount) - 0,MAX(loans.rate) - 0,MAX(payments.payment_amount) - 0,MEAN(loans.loan_amount) - 0,MEAN(loans.rate) - 0,MEAN(payments.payment_amount) - 0,YEAR(LAST(loans.loan_end)),YEAR(LAST(loans.loan_start)),YEAR(LAST(payments.payment_date))
client_id,,,,,,,,,,,,,,,,,,,,,
46180,43851,562,11,10.688553,154017,3834,10887,other,1.38,0,...,437.0,14081.0,9.26,2660.0,7700.850000,3.502500,1186.550336,2016,2014,2015
46109,172677,527,4,12.059178,179032,10853,11961,other,2.82,0,...,102.0,14049.0,9.48,2735.0,8951.600000,3.152500,1375.560284,2015,2013,2014
35214,95849,696,8,11.470529,129124,9389,10336,home,1.40,0,...,1160.0,14767.0,8.44,2874.0,7173.555556,3.108333,1076.987952,2016,2014,2014
32726,235705,730,5,12.370336,126032,5325,10858,other,2.83,1,...,876.0,14802.0,9.10,2695.0,6633.263158,3.058947,944.298013,2016,2014,2014
25707,211422,621,10,12.261611,159279,2203,10363,home,7.40,0,...,566.0,13913.0,9.44,2704.0,7963.950000,3.477000,1178.552795,2015,2014,2014
49545,104564,770,11,11.557555,205786,7061,11514,home,4.23,1,...,1131.0,14971.0,6.52,2873.0,10289.300000,2.684000,1481.188482,2015,2014,2015
48177,190632,769,6,12.158100,141063,659,10684,credit,1.52,1,...,1308.0,14740.0,10.89,2724.0,7424.368421,3.938947,1199.715190,2017,2014,2015
35089,131176,771,3,11.784295,138784,773,11595,other,7.63,1,...,957.0,13194.0,7.63,2563.0,6939.200000,3.513500,1094.748428,2016,2014,2015
39505,153873,610,10,11.943883,148481,9600,10966,cash,0.25,0,...,698.0,14575.0,9.91,2764.0,7424.050000,3.190500,1162.026490,2017,2014,2015
